In [21]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from torch import optim, nn

from plotting import plot_model_outputs

import beer

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [22]:
mean = np.array([3., 3.])
cov = np.array([[2., 1.5], [1.5, 2.]])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Linear subspace',
    width=400,
    height=400,
)
fig.circle(data[:, 0], data[:, 1])
show(fig)

In [87]:
obs_dim = 2
latent_dim = 2
nb_samples = 10

enc_struct = nn.Sequential(
    nn.Linear(obs_dim, 2),
    nn.Tanh(),
    nn.Linear(2, 2),
    nn.Tanh()
)
encoder = beer.models.MLPNormalIso(enc_struct, 2, latent_dim)

dec_struct = nn.Sequential(
    nn.Linear(latent_dim, 2),
    nn.Tanh(),
    nn.Linear(2, 2),
    nn.Tanh()
)
decoder = beer.models.MLPNormalIso(dec_struct, 2, obs_dim)

#latent_model = beer.models.NormalDiagonalCovariance.create(
#    latent_dim,
#    mean_prec=1e2,
#    prec_shape=1e-3,
#    prec_rate=1e-3
#)
latent_model = beer.models.NaturalIsotropicGaussian(2)

model = beer.models.VAE(encoder, decoder, latent_model, nb_samples)

In [88]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
history = beer.inference.History(report_interval=250)

In [93]:
beer.inference.run_training(data, model, optimizer, 1000, history, batch_size=20, lrate_latent_model=1e-1)

Epoch: 4250 	elbo: -16.635381 llh: -288.444988 kld: 44.262618
Epoch: 4500 	elbo: -16.439620 llh: -284.662945 kld: 44.129457
Epoch: 4750 	elbo: -16.517185 llh: -286.437187 kld: 43.906512
Epoch: 5000 	elbo: -16.467471 llh: -286.069103 kld: 43.280307


In [94]:
plot_model_outputs(model, data)

ValueError: not enough values to unpack (expected 4, got 3)

In [95]:
history.plot()

In [96]:
print(model.latent_model.exp_mean)
print(1/model.latent_model.exp_precision)
print(model.latent_model.exp_log_precision)

AttributeError: 'NaturalIsotropicGaussian' object has no attribute 'exp_mean'

In [ ]:
np.log(181)